In [4]:
import os
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler

In [5]:
df = pd.read_csv("../static_merged_data.csv")
x = df.drop(labels="status", axis=1)
y = df[["status"]]
min_max_scale = MinMaxScaler()
x_scaled = min_max_scale.fit_transform(x)

def prepare_train_test_set(X, Y, num_classes):
    X_train, X_test, y_train, y_test = train_test_split(X, Y, shuffle=True, test_size=.2)
    train_labels = y_train.status.astype('category').cat.codes.to_numpy()
    test_labels = y_test.status.astype('category').cat.codes.to_numpy()
    return (X_train, X_test), (train_labels, test_labels)

In [6]:
(X_train, X_test), (train_labels, test_labels) = prepare_train_test_set(x_scaled,y,3)

In [1]:
from keras.models import Model
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import Input, InputLayer
import tensorflow as tf
from keras import Sequential
from keras.layers import Activation 
import tensorflow_model_optimization as tfmot
model = tfmot.quantization.keras.quantize_annotate_model(Sequential([
    InputLayer(input_shape=(4,), name="input_layer"),
    tfmot.quantization.keras.quantize_annotate_layer(Dense(64, activation="relu")),
    tfmot.quantization.keras.quantize_annotate_layer(Dense(128, activation="relu")),
    tfmot.quantization.keras.quantize_annotate_layer(Dense(256, activation="relu")),
    tfmot.quantization.keras.quantize_annotate_layer(Dense(128, activation="relu")),
    tfmot.quantization.keras.quantize_annotate_layer(Dense(16, activation="relu")),
    Dense(3, name="output_layer")
], name="sven_quant_model"))

model.summary()

Model: "sven_quant_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_annotate (Quantize  (None, 64)               320       
 Annotate)                                                       
                                                                 
 quantize_annotate_1 (Quanti  (None, 128)              8320      
 zeAnnotate)                                                     
                                                                 
 quantize_annotate_2 (Quanti  (None, 256)              33024     
 zeAnnotate)                                                     
                                                                 
 quantize_annotate_3 (Quanti  (None, 128)              32896     
 zeAnnotate)                                                     
                                                                 
 quantize_annotate_4 (Quanti  (None, 16)          

In [7]:
from keras.models import load_model
import tensorflow_model_optimization as tfmot

with tfmot.quantization.keras.quantize_scope():
    model = load_model("./sven_quant_model.h5")

In [2]:
from keras.losses import CategoricalCrossentropy, SparseCategoricalCrossentropy
from keras.optimizers import Adam



model.compile(optimizer=Adam(), loss=SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=(model.name + ".h5"),
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

history = model.fit(X_train, train_labels, epochs=50, validation_split=0.1, callbacks=[model_checkpoint_callback])

NameError: name 'X_train' is not defined

In [12]:
import tensorflow_model_optimization as tfmot
quantize_model = tfmot.quantization.keras.quantize_model
# q_aware stands for quantization aware
q_aware_model = quantize_model(model)

# Use `quantize_apply` to actually make the model quantization aware.
with tfmot.quantization.keras.quantize_scope():
    quant_aware_model = tfmot.quantization.keras.quantize_apply(model)

quant_aware_model.summary()

# 'quantized_model' requires a recompile
quant_aware_model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
#q_aware_model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])



Model: "sven_quant_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer_1 (QuantizeL  (None, 4)                3         
 ayer)                                                           
                                                                 
 quant_dense_5 (QuantizeWrap  (None, 64)               325       
 perV2)                                                          
                                                                 
 quant_dense_6 (QuantizeWrap  (None, 128)              8325      
 perV2)                                                          
                                                                 
 quant_dense_7 (QuantizeWrap  (None, 256)              33029     
 perV2)                                                          
                                                                 
 quant_dense_8 (QuantizeWrap  (None, 128)         

In [13]:
train_subset = X_train[0:20000]      # out of 60000
train_labels_subset = train_labels[0:20000]

quant_aware_model.fit(train_subset, train_labels_subset,batch_size=256,epochs=80, validation_split=0.1)

Epoch 1/80
71/71 [==============================] - 2s 9ms/step - loss: 0.1272 - accuracy: 0.9518 - val_loss: 0.1019 - val_accuracy: 0.9580
Epoch 2/80
71/71 [==============================] - 0s 6ms/step - loss: 0.0847 - accuracy: 0.9667 - val_loss: 0.0937 - val_accuracy: 0.9665
Epoch 3/80
71/71 [==============================] - 0s 6ms/step - loss: 0.0861 - accuracy: 0.9646 - val_loss: 0.0799 - val_accuracy: 0.9685
Epoch 4/80
71/71 [==============================] - 0s 6ms/step - loss: 0.0738 - accuracy: 0.9711 - val_loss: 0.0749 - val_accuracy: 0.9700
Epoch 5/80
71/71 [==============================] - 0s 6ms/step - loss: 0.0769 - accuracy: 0.9687 - val_loss: 0.0846 - val_accuracy: 0.9660
Epoch 6/80
71/71 [==============================] - 0s 6ms/step - loss: 0.0695 - accuracy: 0.9727 - val_loss: 0.0760 - val_accuracy: 0.9695
Epoch 7/80
71/71 [==============================] - 0s 6ms/step - loss: 0.0626 - accuracy: 0.9732 - val_loss: 0.0622 - val_accuracy: 0.9740
Epoch 8/80
71/71 [==

In [14]:
_, baseline_model_accuracy = model.evaluate(X_test,test_labels, verbose=0)
_, quant_aware_model_accuracy = quant_aware_model.evaluate(X_test,test_labels,verbose=0)
#_, q_aware_model_accuracy = q_aware_model.evaluate(X_test,test_labels,verbose=0)

with tfmot.quantization.keras.quantize_scope():
  loaded_model = tf.keras.models.load_model("newest_shit.tflite")

#_, newest_aware_model_accuracy = loaded_model.evaluate(X_test,test_labels,verbose=0)

print("Baseline test accuracy: ", baseline_model_accuracy)
print("Quantized test accuracy: ", quant_aware_model_accuracy)
#print("Full Quantized test accuracy: ", newest_aware_model_accuracy)

Baseline test accuracy:  0.9813181161880493
Quantized test accuracy:  0.9870264530181885


In [15]:
def representative_dataset():
    for sample in X_train[:500,:]:
      yield [sample.astype(np.float32)]

In [ ]:
# quantize with 8int weights and 16int activations
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(quant_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.EXPERIMENTAL_TFLITE_BUILTINS_ACTIVATIONS_INT16_WEIGHTS_INT8]
tflite_16int_quant_model = converter.convert()



INFO:tensorflow:Assets written to: C:\Users\sandr\AppData\Local\Temp\tmp8rqp5vmr\assets


INFO:tensorflow:Assets written to: C:\Users\sandr\AppData\Local\Temp\tmp8rqp5vmr\assets


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(quant_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
quantized_tflite_model = converter.convert()
quantized_tflite_file = "own_quantized_aware_model.tflite"

# with open(quantized_tflite_file, "wb") as f:
#     f.write(quantized_tflite_model)

INFO:tensorflow:Assets written to: C:\Users\sandr\AppData\Local\Temp\tmpkvawh20x\assets


INFO:tensorflow:Assets written to: C:\Users\sandr\AppData\Local\Temp\tmpkvawh20x\assets
c:\Users\sandr\source\repos\Monitoring_Converter_ML\venv\lib\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


In [ ]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(quant_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
tflite_quant_model_int8 = converter.convert()

# with open("full_int8_model.tflite", "wb") as f:
#     f.write(tflite_quant_model_int8)

INFO:tensorflow:Assets written to: C:\Users\sandr\AppData\Local\Temp\tmp736vgs0q\assets


INFO:tensorflow:Assets written to: C:\Users\sandr\AppData\Local\Temp\tmp736vgs0q\assets
c:\Users\sandr\source\repos\Monitoring_Converter_ML\venv\lib\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


In [ ]:
import numpy as np

def evaluate_model(interpreter):
    input_index = interpreter.get_input_details()[0]["index"]
    output_index = interpreter.get_output_details()[0]["index"]

    # Run predictions on over y image in the test dataset
    prediction_digits = []
    for i, test_image in enumerate(X_test):
        if i % 1000 == 0:
            print("Evaluated on {n} results so far.".format(n=i))
        
        # Pre-processing: add batch dimension and convert to float32 to match with the model's input data format
        test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
        interpreter.set_tensor(input_index, test_image)

        # Run inference
        interpreter.invoke()

        # Post-processing: remove batch dimension and find the digit with highest probability
        output = interpreter.tensor(output_index)
        digit = np.argmax(output()[0])
        prediction_digits.append(digit)

    print("\n")

    # Compare prediction result with ground truth labels to calculate accuracy
    prediction_digits = np.array(prediction_digits)
    accuracy = (prediction_digits == test_labels).mean()
    return accuracy

In [ ]:
interpreter = tf.lite.Interpreter(model_content=tflite_16int_quant_model)
interpreter.allocate_tensors()
interpreter_default = tf.lite.Interpreter(model_content=quantized_tflite_model)
interpreter_default.allocate_tensors()

test_accuracy = evaluate_model(interpreter)
test_accuracy_default = evaluate_model(interpreter_default)

print("With 16int activations TFLite test_accuracy: ", test_accuracy)
print("Default TFLite test_accuracy: ", test_accuracy_default)

Evaluated on 0 results so far.
Evaluated on 1000 results so far.
Evaluated on 2000 results so far.
Evaluated on 3000 results so far.
Evaluated on 4000 results so far.
Evaluated on 5000 results so far.
Evaluated on 6000 results so far.
Evaluated on 7000 results so far.
Evaluated on 8000 results so far.
Evaluated on 9000 results so far.
Evaluated on 10000 results so far.
Evaluated on 11000 results so far.
Evaluated on 12000 results so far.
Evaluated on 13000 results so far.
Evaluated on 14000 results so far.
Evaluated on 15000 results so far.
Evaluated on 16000 results so far.
Evaluated on 17000 results so far.
Evaluated on 18000 results so far.
Evaluated on 19000 results so far.


Evaluated on 0 results so far.
Evaluated on 1000 results so far.
Evaluated on 2000 results so far.
Evaluated on 3000 results so far.
Evaluated on 4000 results so far.
Evaluated on 5000 results so far.
Evaluated on 6000 results so far.
Evaluated on 7000 results so far.
Evaluated on 8000 results so far.
Evaluate

In [ ]:
with open("sven_quant_act16int_weights8int.tflite", "wb") as f:
    f.write(tflite_16int_quant_model)

with open("sven_quant_full_int8.tflite", "wb") as t:
    t.write(tflite_quant_model_int8)

In [ ]:
with open("sven_quant_int8.tflite", "wb") as m:
    m.write(quantized_tflite_model)